# Implementación del PSO lbest
Se comienza por importar las librerias con algunas utilidades como manejo de arreglos, numeros aleatorios y funciones de prueba

In [1]:
# Manejo de arreglos y operaciones matematicas
import numpy as np

# Importar funciones
import import_ipynb
from utils import FuncionesObjetivo as f

# Numeros aleatorios
import random

# Para visualizar las animaciones
%matplotlib notebook

importing Jupyter notebook from /Users/isaacg/Desktop/Maestría/Primer_Semestre/Algoritmos_Bioinspirados/PSO/utils/FuncionesObjetivo.ipynb


## Implementacion de funciones
Ahora se implementan funciones que se usaran en la implementacion del algoritmo

In [ ]:
def crear_vecindarios(X, k):
    """
    Esta funcion crea los vecindarios de k vecinos para las n particulas
    - input:
        - X: poblacion
        - k: numero de vecinos de cada particula
    """
    # Se 
    # Checar si el numero de vecinos es valido
    if (k % 2 != 0) or k > len(X):
        print("Ingrese un número de vecinos valido (par y menor o igual al numero de particulas)")
        return
    else:
        vecindarios = []
        # Crear vecindarios
        for i in range(len(X)):
            vecindarios.append(np.arange(i-k/2, i+k/2+1))  # Los vecinos son los k/2 anteriores y los k/siguientes
        vecinos = np.array(vecindarios).astype(int)
        # Poner todos los indices dentro del rango valido para los indices
        vecinos[vecinos > len(X) -1] -= len(X)
        return vecinos

## Definir Hiperparametros del algoritmo

In [2]:
# ------------------ Del Problema ------------------
d = 2  # Numero de dimensiones
funcion = f.f_schwefel # funcion objetivo
rango = [[-500, 500]]  # Rango de las variables
mark = [420.96, 420.96, 0]  # Minimo real (si se conoce)

# ------------------ Del Algoritmo ------------------
n = 10  # Numero de particulas
k = 2  # Numero de vecinos (debe ser par y menor o igual a n)
w = 0.9  # Factor de inercia
c1 = 0.5  # Peso cognitivo
c2 = 0.3  # Peso social
max_gen = 100  # Maximo de generaciones
tol = 1e-6  # Tolerancia para detener el algoritmo

# ------------------ Del visualizacion ------------------
animacion = True  # Realizar animacion o no
levels = 20 # numero de curvas de nivel

In [3]:
# Si solo hay un rango se considera el mismo rango para todas las variables
if len(rango) == 1:
    rango *= d
# Convertir rango a np.array
rango = np.array(rango)

# Definimos un mejor fitness grande para iniciar el algoritmo
mejor_fitness= np.infty
gen = 0

# Crear un vetor de soluciones para cada dimension con su rango dado
X = np.random.uniform(low=rango[:, 0], high=rango[:,1], size=((n, d)))

In [5]:
X

array([[-347.73004409, -106.24526874],
       [ 468.9966015 ,  320.87735722],
       [  98.10828679, -389.71745342],
       [ 434.93492748, -255.41030311],
       [-195.97867286,   61.77356315],
       [ 162.14823576, -274.80632331],
       [ -48.70347323,  312.51498881],
       [-485.89062956,  437.00221159],
       [ -36.71940492, -158.70562802],
       [-316.30497722,  457.69830338]])

In [10]:
m

array([[-1,  0,  1],
       [ 0,  1,  2],
       [ 1,  2,  3],
       [ 2,  3,  4],
       [ 3,  4,  5],
       [ 4,  5,  6],
       [ 5,  6,  7],
       [ 6,  7,  8],
       [ 7,  8,  9],
       [ 8,  9,  0]])